# Skrypt do predykcji danych demograficznych oparty o dane historyczne GUS

## Zaimportowanie bibliotek

In [2]:
import requests as rq

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error

from ipywidgets import interact, Checkbox, VBox
from IPython.display import display


## Zaimportowanie danych z API GUS

Zaimportowanie danych z API GUS Portal z API BDL.

In [ ]:
data = {
    "Area": [
        "Polska", "Małopolskie", "Śląskie", "Lubuskie", "Wielkopolskie",
        "Zachodniopomorskie", "Dolnośląskie", "Opolskie", "Kujawsko-Pomorskie",
        "Pomorskie", "Warmińsko-Mazurskie", "Łódzkie", "Świętokrzyskie",
        "Lubelskie", "Podkarpackie", "Podlaskie", "Mazowieckie"
    ],
    "Code": [
        "000000000000", "011200000000", "012400000000", "020800000000",
        "023000000000", "023200000000", "030200000000", "031600000000",
        "040400000000", "042200000000", "042800000000", "051000000000",
        "052600000000", "060600000000", "061800000000", "062000000000",
        "071400000000"
    ]
}

data2 = {
    "category_code": [
        "ludnosc_na_1_km2", #1
        "ludnosc_w_tysiacach",#2
        "wskaznik_urbanizacji",#3
        "liczba_ludnosci_ogolem",#4
        "liczba_ludnosci_ogolem_mezczyzni",#5
        "liczba_ludnosci_ogolem_kobiety",#6
        "mediana_wieku_ludnosci_ogolem",#7
        "mediana_wieku_ludnosci_mezczyzni",#8
        "mediana_wieku_ludnosci_kobiety",#9
        "ludnosc_w_wieku_nieprodukcyjnym_na_100_osob_w_wieku_produkcyjnym",#10
        "wspolczynnik_obciazenia_demograficznego_osobami_starszymi",#11
        "zgony_ogolem",#12
        "zgony_mezczyzni_ogolem",#13
        "zgony_kobiety_ogolem",#14
        "urodzenia_zywe_na_1000_ludnosci",#15
        "zgony_na_1000_ludnosci",#16
        "przyrost_naturalny_na_1000_ludnosci",#17
    ],
    "values": [
        60559,#1
        1645341,#2
        1725015,#3
        72305,#4
        72300,#5
        72295,#6
        746289,#7
        746290,#8
        746291,#9
        60563,#10
        634067,#11
        3231,#12
        30149,#13
        30165,#14
        450540,#15
        450541,#16
        450551,#17
    ]
}


#nie działa do końca
checkboxes = [Checkbox(description=area, value=True) for area in data["Area"]]
checkboxes_container = VBox(checkboxes)

display(checkboxes_container)

def handle_checkbox_change(change):
    selected_areas = [checkbox.description for checkbox in checkboxes if checkbox.value]
    print("Selected areas:", selected_areas)

for checkbox in checkboxes:
    checkbox.observe(handle_checkbox_change, names='value')

In [ ]:
single_frames = []
my_params = {'lang': 'pl', 'format': 'json', 'var-id': None}
for area, code in zip(data['Area'], data['Code']):
    area_frames = f"{area}".replace(" ", "_")
    locals()[area_frames] = []
    for category_code, value in zip(data2['category_code'], data2['values']):
        my_params['var-id'] = value
        response = rq.get(f'https://bdl.stat.gov.pl/api/v1/data/by-unit/{code}', params=my_params)
        output = response.json()
        df_name = f"{category_code}_{area}".replace(" ", "_")
        locals()[df_name] = pd.DataFrame(output['results'][0]['values'], columns=['year', 'val'])
        locals()[df_name] = locals()[df_name].rename(columns={"val":f"{category_code}"})
        locals()[area_frames].append(locals()[df_name])
        print(f"Pobieranie danych: {category_code} dla: {area}") #poprawić liczba pobranych z całości (Zaimportowano x z 289)
    single_frames.append(locals()[area_frames])

In [4]:
frames = {}
for index, area in enumerate(single_frames):
    wframe = pd.DataFrame()
    for frame in area:
        if wframe.empty:
            wframe = frame
        else:
            wframe = wframe.merge(frame, on='year', how='outer')
    
    area_name = data["Area"][index]
    frames[area_name] = wframe

In [ ]:
print(frames)

## Korelacja danych

Tworzymy diagram korelacji dla naszych danych.

In [ ]:
for frame in frames:
    corelacion_data = frames[frame].corr()
    plt.figure(figsize=(16,14))
    sns.heatmap(corelacion_data, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title(f"Diagram korelacji dla {frame}")
    plt.show()

## Wybór i tworzenie modelu

### Drzewa decyzyjne

Tworzenie drzew decyzyjnych dla każdego województwa

In [ ]:
predictions = {}
for key,df in frames.items():
    print("Drzewo decyzyjne dla: ", key)
    frame = df.drop(columns="year", axis = 1)
    frame = frame.fillna(frame.mean())
    x = frame.drop("liczba_ludnosci_ogolem", axis = 1)
    y= frame["liczba_ludnosci_ogolem"]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    data = {"original data": y_test, "predictions": prediction}
    df = pd.DataFrame(data)
    predictions[key] = df
    mse = mean_squared_error(y_test, prediction)
    print(f'Mean Squared Error: {mse}')
    plt.figure(figsize=(20, 10), dpi=600)
    plot_tree(model, filled=True, feature_names=x.columns)
    plt.show()

## Ewaluacja modelu

Ocenimy teraz jakość naszych modeli.

In [ ]:
print(len(predictions))
for key,df in predictions.items():
    df["error"] = df["original data"]-df["predictions"]
    df =df.sort_index()
    print (df)
    print(f"Średni błąd: {df['error'].mean()}")
    positions = range(len(df))
    x = df.index
    y_original = df['original data']
    y_predicted = df['predictions']
    plt.figure(figsize=(10, 6))
    plt.plot(x, y_original, marker='o', linestyle='-', color='b', label='Original')
    plt.plot(x, y_predicted, marker='s', linestyle='--', color='r', label='Predicted')
    plt.xlabel('X-axis label')
    plt.ylabel('Y-axis label')
    plt.title(f'Original vs. Predicted Data {key}')
    plt.legend()
    plt.grid(True)
    plt.show()